In [1]:
from sqlalchemy import create_engine

# Put your current path directory here for assets folder.
static_folder = "/Users/mac/Desktop/MoLVC Internship/Data-management-system-ministry/processing/templates/assets"

# driver test path:
driver_path = "/Users/mac/Documents/chromedriver-mac-x64/chromedriver"

# path of streamlit file (Ex: streamlit_app.py file)
psf = "/Users/mac/Desktop/MoLVC Internship/Data-management-system-ministry/streamlit_app.py"

# add to directory of file for thyda streamlit testing
thyda_dir = "/Users/mac/Desktop/MoLVC Internship/Data-management-system-ministry/processing/visualizations/test"

# file test
fltest = r"some_asean_and_european_currencies_against_khmer_riel_end_period.csv"

your_host = 'localhost'
your_user = 'root'
your_password = 'LaySENG./333'
your_database = 'MoLVT'
your_db_table = 'indonesia_inflation_sample'
your_db_condition = 'Year = 2022'

engine = create_engine(url=f"mysql+pymysql://{your_user}:{your_password}@{your_host}/{your_database}",
                       pool_recycle=3600)

path_file_DB = "/Users/mac/Desktop/MoLVC Internship/Data-management-system-ministry/processing/scrape/IndoInflation/DB_Data/"


In [2]:
import mysql.connector as conn
import pandas as pd

class Database:
    def __init__(self, host, password, user, table=None, database=None):
        """
        Initialize the Database object with the host and password.

        Args:
            host (str): The hostname or IP address of the MySQL server.
            password (str): The password for the MySQL server.

        """
        self.host = host
        self.password = password
        self.user = user
        self.database = database
        self.table = table

    def connection(self):
        """
        Establish a connection to the MySQL server.

        Returns:
            mysql.connector.MySQLConnection: A connection to the MySQL server.
            mysql.connector.cursor.MySQLCursor: A cursor object for executing SQL queries.

        """
        database = self.database
        mydb = conn.connect(host=self.host, user=self.user, passwd=self.password, database=database)
        cursor = mydb.cursor()
        return mydb, cursor

    def create_database(self, database_name):
        """
        Create a new MySQL database if it doesn't exist.

        Args:
            database_name (str): The name of the database to be created.

        """
        mydb, cursor = self.connection()

        self.database = database_name

        # Check if the database exists
        cursor.execute("SHOW DATABASES LIKE %s", (database_name))
        database_exists = cursor.fetchone()
        if not database_exists:
            # Create the database if it doesn't exist
            cursor.execute(f"CREATE DATABASE {database_name}")
            print(f"Database '{database_name}' created successfully.")
        else:
            print(f"Database '{database_name}' already exists.")

        # Close the cursor and the connection
        cursor.close()
        mydb.close()

    def create_table(self, table_name=None):
        """
        Create a MySQL table if it doesn't exist.

        Args:
            table_name (str): The name of the table to create.

        """
        mydb, cursor = self.connection()

        if table_name is None:
            # If table_name is not provided, use the currently set table_name
            table_name = self.table
        # Define the SQL statement to create a table if it doesn't exist
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            No INT AUTO_INCREMENT PRIMARY KEY,
            Country VARCHAR(255),
            Source VARCHAR(255),
            UpdateFrequency VARCHAR(255),
            Status VARCHAR(255),
            Year INT,
            Month  VARCHAR(255),
            Value FLOAT,
            AccessDate DATETIME,
            PublishDate DATE NULL DEFAULT NULL,
            Link VARCHAR(255),
            Note VARCHAR(255)
        )
        """
        # Execute the SQL statement to create the table
        cursor.execute(create_table_query)
        print(f"Table '{table_name}' created successfully or already exists.")

        # Don't forget to commit the changes and close the connection when you're done
        mydb.commit()
        cursor.close()
        mydb.close()

    def insert_data(self, df):
        """
        Insert data from a DataFrame into a MySQL table.

        Args:
            df (pandas.DataFrame): The DataFrame containing the data to be inserted
        """
        mydb, cursor = self.connection()

        inserted_count = 0
        skipped_count = 0

        table_name = self.table

        try:
            # Iterate through the DataFrame and insert values into the table
            for index, row in df.iterrows():

                publish_date = row['Publish Date'] if not pd.isna(row['Publish Date']) else pd.to_datetime(0).date()
                # Handle NaN values in the 'Values' column
                inflation_values = row["Value"] if not pd.isna(row["Value"]) else 0

                # Check for existing entries with the same 'Country', 'Year', 'Month', and 'Value'
                select_query = f"""
                                SELECT COUNT(*) FROM {table_name}
                                WHERE Country = '{row['Country']}' AND Year = '{row['Year']}'
                                AND Month = '{row["Month"]}'
                                """
                cursor.execute(select_query)
                count = cursor.fetchone()[0]

                if count == 0:
                    insert_query = f"""
                             INSERT INTO {table_name} (Country, Source, UpdateFrequency, Status, Year, Month, Value, 
                             AccessDate, PublishDate, Link, Note) VALUES ( '{row['Country']}', '{row['Source']}',
                              '{row['Update frequency']}','{row['Status']}', '{row['Year']}', '{row['Month']}',
                             '{inflation_values}',NOW(), '{publish_date}', '{row['Link']}', '{row['Note']}')
                             """

                    cursor.execute(insert_query)
                    inserted_count += 1
                else:
                    skipped_count += 1

            # Commit the changes and close the connection
            mydb.commit()
            print(f"Inserted {inserted_count} records into '{table_name}' successfully.")
            if skipped_count > 0:
                print(f"Skipped {skipped_count} records as they already exist in '{table_name}'.")
        finally:
            # Always close the cursor and the connection, even in case of exceptions
            cursor.close()
            mydb.close()

    def delete_table(self, table_name=None):
        """
        Delete a MySQL table.

        Args:
            table_name (str): The name of the table to be deleted.

        """
        mydb, cursor = self.connection()

        if table_name is None:
            # If table_name is not provided, use the currently set table_name
            table_name = self.table

        try:
            # Check if the table exists
            cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
            print(f"Table '{table_name}' deleted successfully.")
        except conn.Error as e:
            print(f"Error deleting table '{table_name}': {str(e)}")
        finally:
            # Always close the cursor and the connection, even in case of exceptions
            cursor.close()
            mydb.close()

    def show_table(self, table_name=None):
        """
        Retrieve and display data from the current table.

        """
        mydb, cursor = self.connection()

        if table_name is None:
            # If table_name is not provided, use the currently set table_name
            table_name = self.table
        try:
            # Select all records from the table
            select_query = f"SELECT * FROM {table_name}"
            cursor.execute(select_query)

            # Fetch all records
            records = cursor.fetchall()

            # Get the column names
            column_names = [desc[0] for desc in cursor.description]

            # Display the column names
            print("Table Columns:")
            for col_name in column_names:
                print(col_name, end="\t")
            print("\n-----------------------------------")

            # Display the table data
            for row in records:
                for value in row:
                    print(value, end="\t")
                print()  # Move to the next line for the next row

        except conn.Error as e:
            print(f"Error fetching data from table '{table_name}': {str(e)}")
        finally:
            # Always close the cursor and the connection, even in case of exceptions
            cursor.close()
            mydb.close()

    def read_database(self, your_condition, Indicator='Inflation', unit='percentage', title='Inflation rate'):

        your_table = self.table

        # Define your SQL query
        sql_query = f"SELECT * FROM {your_table} WHERE {your_condition}"

        # Connect to the database
        mydb, cursor = self.connection()

        # Use pandas.read_sql() to read data into a DataFrame
        df = pd.read_sql(sql_query, mydb)

        # Close the database connection
        mydb.close()

        base_col = ['No.', 'Title', 'Country', 'Source', 'Update frequency', 'Status',
                    'Year', 'Month', 'Indicator', 'Sub 1', 'Sub 2', 'Sub 3', 'Sub 4',
                    'Sub 5', 'Sub 6', 'Unit', 'Value', 'Access Date', 'Publish Date',
                    'Link (if available)', 'Note', 'Note.1']

        df.rename(columns={'No': 'No.', 'AccessDate': 'Access Date', 'PublishDate': 'Publish Date',
                           'Link': 'Link (if available)', 'UpdateFrequency': 'Update frequency'}, inplace=True)

        missing_cols = [col for col in base_col if col not in df.columns]
        nrow = df.shape[0]
        for i, col in enumerate(missing_cols):
            df[col] = \
                [[title] * nrow, [Indicator] * nrow, ['.'] * nrow,
                 ['.'] * nrow, ['.'] * nrow, ['.'] * nrow, ['.'] * nrow, ['.'] * nrow, [unit] * nrow, [None] * nrow][i]
        return df[base_col]

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import pandas as pd
import numpy as np
from datetime import datetime

class Webscrap:
    ## The implamentation how to use all function of code below:
    """# Create an instance of the Webscrap class
    webscraper = Webscrap(driver_path='path_to_chromedriver')

    # Scrape Thai Price Consumer data and print a summary
    thai_price_consumer_data = webscraper.Thai_Price_Consomer()
    print("Summary of Thai Price Consumer Data:")


    # Scrape Singapore inflation data and print a summary
    print("\nSummary of Singapore Inflation Data:")
    webscraper.SG_Inflation.py()

    # Scrape Laos inflation data for a specific year (e.g., 2021) and print a summary
    laos_inflation_data = webscraper.LaoInflation(year=2021)
    print("\nSummary of Laos Inflation Data for 2021:")
    """

    # ------------------------------------------------------------------
    def __init__(self, driver_path):
        # self.url = url
        self.driver_path = driver_path
        # self.year = year

    # Initialize a web browser
    def Browser(self, url):
        # Configure Chrome WebDriver in headless mode (optional)
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Enable headless mode
        service = Service(self.driver_path)
        browser = webdriver.Chrome(service=service, options=options)
        browser.get(url)
        return browser

    # Scrape Thai price consumer data from a webpage
    def Thai_Price_Consomer(self):
        url = 'http://www.indexpr.moc.go.th/price_present/cpi/stat/others/report_core1.asp?tb=cpig_index_country&code' \
              '=93&c_index=a.change_year'
        browser = self.Browser(url)
        wait = WebDriverWait(browser, 20)
        table = wait.until(ec.presence_of_element_located((By.XPATH, '/html/body/table[3]')))
        rows = table.find_elements(By.TAG_NAME, value="tr")

        data = []
        start = int(input("Start from: "))
        end = int(input("End at: "))

        n = 0
        for row in rows[3:]:

            cells = row.find_elements(By.TAG_NAME, value="td")

            year = browser.find_elements(
                By.XPATH, 
                "/html/body/table[3]/tbody/tr[4]/td[1]/p/strong/font/font/font")
            
            start_index = start - 2001
            end_index = end - 2001

            for i in range(start_index, end_index):
                # Extract data from table cells
                month = cells[0].text
                Now = datetime.now().strftime('%d/%m/%y')
                source = 'Ministry of Commerce'
                status = 'Real'
                f_update = 'Monthly'
                country = 'Thailand'
                ########################################################
                value = cells[i].text
                ########################################################
                Pub_Date = np.nan
                Link = url
                Note = 'abc'

                # Append data to the list
                data.append([n, country, source, f_update, status, year, month, value,
                             Now, Pub_Date, Link, Note])

        # Map numeric month values to month names
        e = ['January', 'February', 'March', 'April', 'May', 'June',
             'July', 'August', 'September', 'October', 'November', 'December']

        # Create a DataFrame
        column_names = ['No.', 'Country', 'Source', 'Update frequency', 'Status', 'Year',
                        'Month', 'Value', 'Access Date', 'Publish Date', 'Link', 'Note']  # Repeat for other columns
        
        df = pd.DataFrame(data, columns=column_names)
        
        len_no = df.shape[0]        
        df['No.'] = range(1, len_no + 1)
        
        df.to_csv(path_file_DB + 'Thai.csv', index=False)

        # # Map numeric month values to month names
        # dic = dict(zip(df['Month'].to_list(), e))
        # df['Month'] = df['Month'].map(dic)
        # melted_df = pd.melt(df, id_vars=['Month', 'Access Date', 'Source', 'Status', 'Update frequency', 'Country',
        #                                  'Publish Date', 'Link'], var_name='Year of GB', value_name='Value')
        # melted_df[['Note', 'Year']] = melted_df['Year of GB'].str.split('_', expand=True)
        #
        # melted_df.drop(columns=['Year of GB'], inplace=True)
        #
        # # Create a dictionary to map 'GCPI' to 'Headline Consumer Price Index' and 'BCPI' to 'Core Consumer Price Index'
        # note_mapping = {
        #     'GCPI': 'Headline Consumer Price Index',
        #     'BCPI': 'Core Consumer Price Index'
        # }

        # # Use the mapping dictionary to update the 'Note' column
        # melted_df['Note'] = melted_df['Note'].map(note_mapping)
        # # melted_df['Values'] = melted_df['Values'].replace(' ',np.nan)
        
#         return df

    # Scrape Laos inflation data
    def LaoInflation(self, year):
        url = 'https://www.bol.gov.la/en/inflation'
        browser = self.Browser(url)
        select_year = browser.find_element(By.XPATH, '//*[@id="year"]')
        # year = input("Input Year :")
        select_year.send_keys(str(year))
        search_bt = browser.find_element(By.XPATH, '//*[@id="frm_sel"]/div/div[2]/div/button[1]')
        search_bt.click()

        wait = WebDriverWait(browser, 10)
        table = wait.until(ec.presence_of_element_located((By.XPATH, '//*[@id="frm_sel"]/div/div[3]/table')))
        # Create an empty list to store the table data
        data = []

        # Find all rows in the table body
        rows = table.find_elements(By.XPATH, './/tbody/tr')

        # Iterate through the rows
        for row in rows:
            # Find all cells in the current row
            cells = row.find_elements(By.XPATH, './/td')
            # Pub_Date = np.nan
            Now = datetime.now().strftime('%d/%m/%y')
            source = ' BANK OF THE LAO P.D.R'
            status = 'Real'
            f_update = 'Monthly'
            country = 'Lao'
            url = 'https://www.bol.gov.la/en/inflation'

            # Extract and store the cell text in a list
            status = 'Real'
            row_data = [year] + [cell.text for cell in cells] + [np.nan] + [Now] + [source] + [status] + [f_update] + [
                country, url]

            # Append the row data to the data list
            data.append(row_data)

        # Define column names
        column_names = ['Year', 'Note', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                        'September', 'October', 'November', 'December', 'C.Y.Ave.', 'Publish Date', 'Access Date',
                        'Source', 'Status', 'Update frequency', 'Country', 'Link']  # Repeat for other months and columns

        # Create a DataFrame
        data2 = pd.DataFrame(data, columns=column_names)
        melted_df2 = pd.melt(data2,
                             id_vars=['Month', 'Access Date', 'Source', 'Status', 'Update frequency',
                                      'Country', 'Publish Date', 'Link', 'Value', 'Note', 'Year'],
                             var_name='Month',
                             value_name='Value')

        browser.quit()
        return melted_df2


def main():
    
    db = Database(your_host, your_password, your_user, table=your_db_table, database=your_database)
    # db.create_table()

    webscraper = Webscrap(driver_path=driver_path)


    # Scrape Thai Price Consumer data and print a summary
    print(webscraper.Thai_Price_Consomer())

    # # Clean the 'Value' column by removing non-numeric characters and replacing with np.nan
    # thai_price_consumer_data['Value'] = thai_price_consumer_data['Value'].str.replace(r'[^\d.]', '', regex=True)
    # thai_price_consumer_data['Value'] = thai_price_consumer_data['Value'].replace('', np.nan, regex=True)
    #
    # # Convert the 'Value' column to float
    # thai_price_consumer_data['Value'] = thai_price_consumer_data['Value'].astype("float64")

    # print("Summary of Thai Price Consumer Data:", thai_price_consumer_data['Value'])
    # db.insert_data(thai_price_consumer_data)










    # # Scrape Laos inflation data for a specific year (e.g., 2021) and print a summary
    # laos_inflation_data = webscraper.LaoInflation(year=2021)
    # print("\nSummary of Laos Inflation Data for 2021:", laos_inflation_data)
    # # db.insert_data(laos_inflation_data)
    # db.show_table()


main()


Start from: 2022
End at: 2023
None


In [4]:
pd.read_csv(path_file_DB + 'Thai.csv')

,No.,Country,Source,Update frequency,Status,Year,Month,Value,Access Date,Publish Date,Link,Note
0,1,Thailand,Ministry of Commerce,Monthly,Real,[],2545,2566.00,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
1,2,Thailand,Ministry of Commerce,Monthly,Real,[],มกราคม,3.23,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
2,3,Thailand,Ministry of Commerce,Monthly,Real,[],กุมภาพันธ์,5.28,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
3,4,Thailand,Ministry of Commerce,Monthly,Real,[],มีนาคม,5.73,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
4,5,Thailand,Ministry of Commerce,Monthly,Real,[],เมษายน,4.65,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
5,6,Thailand,Ministry of Commerce,Monthly,Real,[],พฤษภาคม,7.10,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
6,7,Thailand,Ministry of Commerce,Monthly,Real,[],มิถุนายน,7.66,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
7,8,Thailand,Ministry of Commerce,Monthly,Real,[],กรกฎาคม,7.61,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
8,9,Thailand,Ministry of Commerce,Monthly,Real,[],สิงหาคม,7.86,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc
9,10,Thailand,Ministry of Commerce,Monthly,Real,[],กันยายน,6.41,28/09/23,NaN,http://www.indexpr.moc.go.th/price_present/cpi...,abc


In [48]:
class Webscrap:
    ## The implamentation how to use all function of code below:
    """# Create an instance of the Webscrap class
    webscraper = Webscrap(driver_path='path_to_chromedriver')

    # Scrape Thai Price Consumer data and print a summary
    thai_price_consumer_data = webscraper.Thai_Price_Consomer()
    print("Summary of Thai Price Consumer Data:")


    # Scrape Singapore inflation data and print a summary
    print("\nSummary of Singapore Inflation Data:")
    webscraper.SG_Inflation.py()

    # Scrape Laos inflation data for a specific year (e.g., 2021) and print a summary
    laos_inflation_data = webscraper.LaoInflation(year=2021)
    print("\nSummary of Laos Inflation Data for 2021:")
    """

    # ------------------------------------------------------------------
    def __init__(self, driver_path):
        # self.url = url
        self.driver_path = driver_path
        # self.year = year

    # Initialize a web browser
    def Browser(self, url):
        # Configure Chrome WebDriver in headless mode (optional)
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Enable headless mode
        service = Service(self.driver_path)
        browser = webdriver.Chrome(service=service, options=options)
        browser.get(url)
        return browser

    # Scrape Thai price consumer data from a webpage
    def Thai_Price_Consomer(self):
        global year
        url = 'http://www.indexpr.moc.go.th/price_present/cpi/stat/others/report_core1.asp?tb=cpig_index_country&code' \
              '=93&c_index=a.change_year'
        browser = self.Browser(url)
        wait = WebDriverWait(browser, 20)
        tbody_table = wait.until(ec.presence_of_element_located((By.XPATH, '/html/body/table[3]/tbody')))
        rows = tbody_table.find_elements(By.TAG_NAME, value="tr")

        # for duration_in_year in range(start_index, end_index):
        months = []
        values = []
        
        start = int(input("Start from: "))
        stop = int(input("End at: "))

        start_index = start - 2001
        end_index = stop - 2001
        
        for row in rows[3:]:
            for index in range(start_index, end_index):
                years = row.find_element(By.XPATH, f"/html/body/table[3]/tbody/tr[{index}]")
                for year in years:
                    print(year.text)
            
#             for i in range()
#             jan = row.find_elements(By.XPATH, "/html/body/table[3]/tbody/tr[5]")
            
#             for i in jan:
                
#                 month = i[0].text
#                 value = i.text
#                 values.append(value)
#                 months.append(month[0].text)
        
    
    
    
    
#         df = pd.DataFrame(data={"month":month, "value":values}, columns=["month", "value"])
#         df.to_csv(path_file_DB + 'test_thai.csv', index=False)

web_scrape = Webscrap(driver_path=driver_path).Thai_Price_Consomer()
web_scrape

Start from: 2022
End at: 2023


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/table[3]/tbody/tr[21]"}
  (Session info: headless chrome=117.0.5938.92)
Stacktrace:
0   chromedriver                        0x00000001018b9638 chromedriver + 5002808
1   chromedriver                        0x00000001018b0a53 chromedriver + 4966995
2   chromedriver                        0x0000000101461a57 chromedriver + 449111
3   chromedriver                        0x00000001014a7d05 chromedriver + 736517
4   chromedriver                        0x00000001014a7ec1 chromedriver + 736961
5   chromedriver                        0x000000010149b9f6 chromedriver + 686582
6   chromedriver                        0x00000001014ce41d chromedriver + 893981
7   chromedriver                        0x000000010149b8b8 chromedriver + 686264
8   chromedriver                        0x00000001014ce5ae chromedriver + 894382
9   chromedriver                        0x00000001014e9391 chromedri

In [ ]:
pd.read_csv(path_file_DB + 'test_thai.csv')